In [41]:
import os
from dotenv import load_dotenv
from operator import itemgetter
load_dotenv()

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.document_loaders.csv_loader import CSVLoader

from langchain.docstore.document import Document

from langchain_community.vectorstores import Neo4jVector

from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.memory.chat_message_histories.in_memory import ChatMessageHistory
from langchain.schema import messages_from_dict, messages_to_dict
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain, ConversationChain
import json

In [22]:

embedding_model = OpenAIEmbeddings()
llm =  ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
URL = "neo4j+ssc://ba68e959.databases.neo4j.io"
USERNAME = "neo4j"
PASSWORD = os.getenv('NEO4J_PASSWORD') 

original_chain = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

In [63]:
import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI


genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
# embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embedding_model = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)
llm = ChatGoogleGenerativeAI(model="gemini-pro")
URL = "neo4j+ssc://ba68e959.databases.neo4j.io"
USERNAME = "neo4j"
PASSWORD = os.getenv('NEO4J_PASSWORD') 

original_chain = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)
print("OK")

OK


In [67]:
docs = [
                Document(
                    page_content="Init vector store.", meta={"source": "knowledge"}
                )
            ]

In [68]:
neo4j_db = Neo4jVector.from_documents(
    docs,
    embedding_model,
    url=URL,
    username=USERNAME,
    password=PASSWORD,
    index_name="knowlege",  # vector by default
    node_label="GeminiKnowlege",  # Chunk by default
)

In [47]:
original_chain.run('what do you know about Python in less than 10 words')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: what do you know about Python in less than 10 words
AI:

> Finished chain.


'Python is a versatile, open-source programming language popular for its simplicity and readability.'

In [48]:
extracted_messages = original_chain.memory.chat_memory.messages
extracted_messages

[HumanMessage(content='what do you know about Python in less than 10 words'),
 AIMessage(content='Python is a versatile, open-source programming language popular for its simplicity and readability.')]

In [49]:
ingest_to_db = messages_to_dict(extracted_messages)
ingest_to_db

[{'type': 'human',
  'data': {'content': 'what do you know about Python in less than 10 words',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'human',
   'name': None,
   'id': None,
   'example': False}},
 {'type': 'ai',
  'data': {'content': 'Python is a versatile, open-source programming language popular for its simplicity and readability.',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'ai',
   'name': None,
   'id': None,
   'example': False}}]

In [50]:
retrieve_from_db = json.loads(json.dumps(ingest_to_db))
retrieve_from_db

[{'type': 'human',
  'data': {'content': 'what do you know about Python in less than 10 words',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'human',
   'name': None,
   'id': None,
   'example': False}},
 {'type': 'ai',
  'data': {'content': 'Python is a versatile, open-source programming language popular for its simplicity and readability.',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'ai',
   'name': None,
   'id': None,
   'example': False}}]

In [51]:
retrieved_messages = messages_from_dict(retrieve_from_db)
retrieved_messages

[HumanMessage(content='what do you know about Python in less than 10 words'),
 AIMessage(content='Python is a versatile, open-source programming language popular for its simplicity and readability.')]

In [52]:
retrieved_chat_history = ChatMessageHistory(messages=retrieved_messages)
retrieved_chat_history

ChatMessageHistory(messages=[HumanMessage(content='what do you know about Python in less than 10 words'), AIMessage(content='Python is a versatile, open-source programming language popular for its simplicity and readability.')])

In [53]:
retrieved_memory = ConversationBufferMemory(chat_memory=retrieved_chat_history)
retrieved_memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='what do you know about Python in less than 10 words'), AIMessage(content='Python is a versatile, open-source programming language popular for its simplicity and readability.')]))

In [54]:
reloaded_chain = ConversationChain(
    llm=llm,
    verbose=True,
    memory=retrieved_memory
)
reloaded_chain.run('what about Javascript and Ruby')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: what do you know about Python in less than 10 words
AI: Python is a versatile, open-source programming language popular for its simplicity and readability.
Human: what about Javascript and Ruby
AI:

> Finished chain.


'JavaScript is a widely-used programming language for dynamic and interactive web content, while Ruby is a general-purpose, dynamic programming language known for its elegance and developer-friendly syntax.'

In [ ]:
# open-ai
vectorstore = Neo4jVector.from_existing_index(
    embedding_model,
    url=URL,
    username=USERNAME,
    password=PASSWORD,
    node_label="Chunk",
    index_name="vector",
)
temp_history = []
vectorstore

In [69]:
# google-ai
vectorstore = Neo4jVector.from_existing_index(
    embedding_model,
    url=URL,
    username=USERNAME,
    password=PASSWORD,
    node_label="GeminiKnowlege",
    index_name="knowlege",
)
temp_history = []
vectorstore

In [70]:
vectorstore.similarity_search("Tôi tên gì?")

[Document(page_content='Init vector store.', metadata={'info': 'Init vector store.'})]

In [71]:

template = """
Given the following extracted parts of a long document and a question, create a final answer,
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.
When providing responses to users, ensure they are presented using only tag in HTML.
{context}
Relevant pieces of previous conversation:
{chat_history}
(You do not need to use these pieces of information if not relevant)
Answer language: Vietnamese
Human: {human_input}
Chatbot:
"""
prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"],
    template=template,
)

# Convert chat history to LangChain message format
retrieved_messages = messages_from_dict(temp_history)
retrieved_chat_history = ChatMessageHistory(messages=retrieved_messages)

# Set up conversation memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="human_input",
    chat_memory=retrieved_chat_history,
)

# Load LangChain model
chain = load_qa_chain(llm, chain_type="stuff", memory=memory, prompt=prompt)

# Process query with LangChain 1
query = "Tôi tên gì?"
docs = vectorstore.similarity_search(query)
rs = chain(
    {"input_documents": docs, "human_input": query},
    return_only_outputs=True,
)
new_chat_history = messages_to_dict(chain.memory.chat_memory.messages)
temp_history =  new_chat_history
rs["output_text"]


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'```html\nTôi không có thông tin đó.\n```'